# export fgrid and fault id files from Petrel 

Make sure these two files are exported using the same setting, including setting cell origin to be (I=0, J=max, K). This is because, in Petrel, i goes from left to right (west to east), j goes from top to bottom (north to south). In CMG, i is the same, but j goes from bottom to top (south to north).

# extract information for each cell from a fgrid file

extract average x, y, z coordinates and fault id for each cell

In [1]:
from fgrid2npy import fgrid2npy
coor_npy = fgrid2npy(
    fgrid_file_path = "data/JD_Sula_2025_gmc.fgrid",
    save_dir = "results/JD_Sula_2025_gmc_coor&fault.npy"
    )

Parsing the fgrid file: data/JD_Sula_2025_gmc.fgrid
Units: METRES  
Processing 125190 coordinate sections...
Total cells processed: 125190
Cells filled: 125190/125190 (100.0%)
Coordinates array shape: (107, 117, 10, 3)
Coordinates array dtype: float64
Coordinate Statistics:
  X range: 424730.20 to 477843.57
  Y range: 7072225.88 to 7129985.88
  Z range: 348.63 to 2846.26
Data saved to results/JD_Sula_2025_gmc_coor&fault.npy


# decompress fault id file and attach it to the cell info dataframe

In [2]:
from CMG_format_decompress import CMG_format_decompress
fault_id = CMG_format_decompress('data/JD_Sula_2025_gmc_fault_id.GRDECL')
print(fault_id.shape)

(125190,)


In [3]:
import numpy as np

i=107; j=117; k=10;
# initialize a nan array for fault id
nan_slice = np.full((i,j,k,1), np.nan)
fault_npy = np.concatenate((coor_npy, nan_slice), axis=3)

# fill the fault id
for ii in range(i):
    for jj in range(j):
        for kk in range(k):
            fault_npy[ii,jj,kk,3] = fault_id[kk*j*i+jj*i+ii]

# fault_npy[:,:,:,3] = fault_npy[:,:,:,3].astype(int)
fault_npy[:,:,:,3].dtype

# flip the j direction, because Petrel count j from top to bottom, and CMG count j from bottom to top.
fault_npy = fault_npy[:,::-1,:,:]

# fill the non-fault id with nan
fault_npy[fault_npy == -999] = np.nan

# save the fault_npy
np.save('results/JD_Sula_2025_gmc_coor&fault.npy', fault_npy)

print(fault_npy.shape)

(107, 117, 10, 4)
